lessons from scratching the code second

- First I just tried to replicate the same code Jeremy wrote(as he and many expert advised), I didn't understand what these or those codes function
- Second, I started to feel myself, inheriting module, which controls the whole forward or backward, was quite reasonable.
- Feels like getting out of numb state.

In [ ]:
from torch import tensor
from fastai import datasets
import gzip, pickle
from math import sqrt

In [ ]:
MNIST_URL = 'http://deeplearning.net/data/mnist/mnist.pkl'

In [3]:
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        (x_train, y_train), (x_valid, y_valid), _ = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

In [5]:
train_x, train_y, valid_x, valid_y = get_data()

In [6]:
del x_train, y_train, x_valid, y_valid
import gc
gc.collect()

171

Dataset normalize
---

In [7]:
train_m, train_s = train_x.mean(), train_x.std()

In [8]:
def normalize(x, m, s): return((x-m)/s)

In [9]:
train_x = normalize(train_x, train_m, train_s)
valid_x = normalize(valid_x, train_m, train_s)

In [10]:
def describe(x): return(f"mean: {x.mean()}, std: {x.std()}, shape:{x.shape}, type: {x.type()}")

In [ ]:
describe(train_x)

'mean: 0.00012300178059376776, std: 1.0, shape:torch.Size([50000, 784]), type: torch.FloatTensor'

In [12]:
describe(valid_x)

'mean: -0.005747819785028696, std: 0.9924333691596985, shape:torch.Size([10000, 784]), type: torch.FloatTensor'

In [ ]:
class Module():
    def __call__(self, *arg):
        self.arg = arg
        self.out = self.forward(*arg) #check if i can put *self.arg
    # def forward(self)

    def backward(self):
        self.bwd(self.out, *self.arg)

In [ ]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b
    def forward(self, x): return x @ self.w + self.b
    def bwd(self,y,x):
        self.w.g = x.t() @ y.g 
        self.b.g = y.g.sum(0)
        x.g = y.g @ self.w.t()

In [93]:
class Mse(Module):
    def forward(self, x, y): return (x.squeeze() - y.float()).pow(2).mean().unsqueeze(-1)
    def bwd(self, out, x, y): x.g =  (x.squeeze() - y.float()).unsqueeze(-1)*2/x.shape[0]

In [ ]:
class Relu(Module):
    def forward(self, x): return x.clamp_min(0.)-0.5
    def bwd(self, y, x): x.g = (x>0).float() * y.g

In [ ]:
class Model():
    def __init__(self, w1, w2, b1, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
    def __call__(self, x, y):
        for l in self.layers: x = l(x)
        return self.loss(x,y)
    def backward(self):
        self.loss.backward()
        for r in reversed(self.layers): r.backward()

In [ ]:
from torch import randn, zeros

In [ ]:
n, nh, c = train_x.shape[1], 68, 1
w1 = randn(n, nh) * sqrt(2/n)
b1 = zeros(nh)
w2 = randn(nh, c) * sqrt(2/nh)
b2 = zeros(c)

In [ ]:
model = Model(w1, w2, b1, b2)

In [ ]:
model(train_x, train_y)

tensor([32.8865])

In [ ]:
model.backward()